In [1]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By

In [11]:
driver = webdriver.Chrome()

In [12]:
# 打开抖音
# driver.get('https://www.douyin.com/search/cpr?publish_time=0&sort_type=0&source=switch_tab&type=video')
driver.get('https://www.douyin.com/search/cpr%E6%95%99%E5%AD%A6?aid=cab22447-dc97-43d9-bb28-f1c528b626fd&publish_time=0&sort_type=0&source=normal_search&type=video')
time.sleep(2)

In [4]:
df = pd.DataFrame(columns=['Published_Name', 'Title', 'link'])

In [13]:
RECORD_TIME = 300   # 记录词条数量

In [14]:
# 获取初始内容数量
content_elements = driver.find_elements(By.XPATH, '//li[@class = "aCTzxbOJ pYgrEk__"]')  # 根据实际情况替换为相应的CSS选择器
content_count = len(content_elements)

# 标识是否已滚动到底部
reached_end = False

# 模拟滚动，直到内容数量达到300条
while content_count < RECORD_TIME and not reached_end:
    # 记录滚动前的页面高度
    prev_scroll_height = driver.execute_script('return document.body.scrollHeight;')

    # 模拟向下滚动
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(2)  # 适当暂停，等待页面加载

    # 检查页面是否已滚动到底部
    new_scroll_height = driver.execute_script('return document.body.scrollHeight;')
    if new_scroll_height == prev_scroll_height:
        # 如果页面高度没有变化，说明已经滚动到底部
        reached_end = True
        break

    # 再次获取内容数量
    content_elements = driver.find_elements(By.XPATH, '//li[@class = "aCTzxbOJ pYgrEk__"]')  # 根据实际情况替换为相应的CSS选择器
    content_count = len(content_elements)

if reached_end:
    print('Reached end of page, and only {} items found'.format(content_count))
else:
    print('Reached target count:{}'.format(content_count))

# 处理和爬取内容
temp_index = 0
for element in content_elements:
    # 获取发布者名称
    published_name = element.find_element(By.XPATH, './/span[@class = "OhTcPZd3"]').text
    df.loc[temp_index, 'Published_Name'] = published_name
    # 获取标题
    title = element.find_element(By.XPATH, './/div[@class = "swoZuiEM"]').text
    df.loc[temp_index, 'Title'] = title
    # 获取链接
    link = element.find_element(By.XPATH, './/a').get_attribute('href')
    df.loc[temp_index, 'link'] = link
    
    temp_index += 1

Reached end of page, and only 214 items found


In [15]:
df

,Published_Name,Title,link
0,申庭田田初级护师加油站,护理操作示范视频-CPR#护士 #护士懂护士#护考,https://www.douyin.com/video/7110119828421184775
1,急救大白叔,大白叔解惑：CPR时，胸外按压“怎么按”？ #急救#科普 #心肺复苏 #救人 #热门,https://www.douyin.com/video/7002210199243066665
2,浙江名医馆,心肺复苏（CPR）有哪些要点？快来跟着医护一起学 #心肺复苏 #健康科普 #CPR #教学,https://www.douyin.com/video/7220715900591148348
3,崇州市中医医院,接上期视频，所以“正确的CPR该怎么做？” 以上是我院杨露、周珊、廖小儀在成都护理学会举办的...,https://www.douyin.com/video/7174010618868010255
4,护你一世周全,心肺复苏，一个人人都应该会的抢救生命技能，但愿永远用不上#医学知识科普 #急救 #急救技能,https://www.douyin.com/video/7028796209082928420
...,...,...,...
209,应急救护师资,,https://www.douyin.com/video/6952157281882623263
210,玉玊,心肺复苏技能的示范、练习，绝对禁止在真人身上做！！！,https://www.douyin.com/video/6612620789613071629
211,rcyy2020,瑞成小课堂 #心肺复苏 #急救技能,https://www.douyin.com/video/7005481163515006244
212,晓晓晓星,人人学急救，急救为人人。急救小课堂开课啦！全新的急协为你带来的cpr操作步骤展示。生命无彩排...,https://www.douyin.com/video/7170681336825400580


In [16]:
driver.close()

In [17]:
df.to_csv('douyin.csv', index=False, encoding='utf-8-sig')